In [31]:
import pandas as pd
import numpy as np


In [32]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [33]:
dataset=pd.read_csv("Insurance.csv")

In [34]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [35]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [36]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [37]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [38]:
independent=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]
dependent=dataset[['charges']]

In [39]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)


In [40]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [41]:
scy=StandardScaler()
y_train=scy.fit_transform(y_train)
y_test=scy.transform(y_test)

In [46]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor


param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.05],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1]
}


regressor = XGBRegressor(eval_metric='rmse', n_jobs=-1, use_label_encoder=False)

grid = GridSearchCV(estimator=regressor,
                    param_grid=param_grid,
                    cv=3,
                    verbose=3,
                    n_jobs=-1)


grid.fit(x_train, y_train)


Fitting 3 folds for each of 32 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:    6.0s finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False,
                                    eval_metric='rmse', gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learn...
                                    n_jobs=-1, num_parallel_tree=None,
                                    objective='reg:squarederror',
                                    predictor=None, random_state=None,
                                    reg_alpha=None, ...),
             iid='warn', n_

In [47]:
re=grid.cv_results_
grid_predictions=grid.predict(x_test)
print("The R_score value for best parameter{}: ".format(grid.best_params_))

The R_score value for best parameter{'colsample_bytree': 1, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1}: 


In [48]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_learning_rate,param_max_depth,param_n_estimators,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.089250,0.007408,0.009453,0.006788,0.8,0.01,3,100,0.8,"{'colsample_bytree': 0.8, 'learning_rate': 0.0...",0.553650,0.605067,0.609016,0.589204,0.025234,31
1,0.110964,0.021432,0.000000,0.000000,0.8,0.01,3,100,1,"{'colsample_bytree': 0.8, 'learning_rate': 0.0...",0.553831,0.631399,0.631163,0.605406,0.036531,29
2,0.385613,0.113859,0.006055,0.006980,0.8,0.01,3,200,0.8,"{'colsample_bytree': 0.8, 'learning_rate': 0.0...",0.767741,0.781325,0.777780,0.775607,0.005756,22
3,0.578729,0.002311,0.007444,0.007945,0.8,0.01,3,200,1,"{'colsample_bytree': 0.8, 'learning_rate': 0.0...",0.764711,0.785238,0.779520,0.776477,0.008654,21
4,0.252824,0.080343,0.010436,0.007380,0.8,0.01,5,100,0.8,"{'colsample_bytree': 0.8, 'learning_rate': 0.0...",0.544641,0.593919,0.598498,0.578981,0.024395,32
5,0.345433,0.024229,0.006877,0.006514,0.8,0.01,5,100,1,"{'colsample_bytree': 0.8, 'learning_rate': 0.0...",0.547071,0.620864,0.621464,0.596411,0.034948,30
6,0.705052,0.001925,0.003723,0.005265,0.8,0.01,5,200,0.8,"{'colsample_bytree': 0.8, 'learning_rate': 0.0...",0.753613,0.766470,0.763636,0.761231,0.005519,24
7,0.672645,0.010926,0.010183,0.009531,0.8,0.01,5,200,1,"{'colsample_bytree': 0.8, 'learning_rate': 0.0...",0.748534,0.768654,0.766649,0.761265,0.009054,23
8,0.341569,0.020637,0.009883,0.007020,0.8,0.05,3,100,0.8,"{'colsample_bytree': 0.8, 'learning_rate': 0.0...",0.829796,0.826422,0.829338,0.828520,0.001494,5
9,0.338904,0.004995,0.009349,0.004939,0.8,0.05,3,100,1,"{'colsample_bytree': 0.8, 'learning_rate': 0.0...",0.830032,0.830666,0.829101,0.829933,0.000643,3


In [49]:
print("✅ Best Score (CV):", grid.best_score_)


✅ Best Score (CV): 0.8346091285847963


In [50]:
y_pred = grid.predict(x_test)
from sklearn.metrics import r2_score
r_score = r2_score(y_test, y_pred)
print("r² Score:", r_score)

r² Score: 0.8939704010409936


In [51]:
import pickle
filename="finalized_model_xgboostingRegression.sav"

In [52]:
pickle.dump(grid.best_estimator_, open(filename, 'wb'))

In [53]:
preinput=sc.transform([[25,20,2,1,1]])

In [54]:
preinput

array([[-1.03919296, -1.72664148,  0.75672907,  1.01127431,  2.00280702]])

In [55]:
loaded_model=pickle.load(open("finalized_model_xgboostingRegression.sav",'rb'))
result=loaded_model.predict(preinput)
result

array([0.308738], dtype=float32)

In [56]:
preoutput=scy.inverse_transform([result])

In [57]:
preoutput

array([[16786.062]], dtype=float32)